In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

# Xarxes convolucionals

L'objectiu d'avui és la creació d'una xarxa convolucional que obtengui com a mínim igual resultat que la xarxa completament connectada implementada la setmana anterior però amb menys paràmetres. Per poder realitzar comparacions directes emprarem el mateix conjunt de dades.

Com objectius secundaris tenim:

1. Aprenentatge de noves estratègies per evitar `overfitting`.
2. Us d'un nou optimitzador.
3. Visualització dels resultats dels filtres convolucionals.

In [ ]:
etiquetes = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

train_batch_size = 64
test_batch_size = 100

# Definim una seqüència (composició) de transformacions 
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # mitjana, desviacio tipica
    ])

# Descarregam un dataset ja integrat en la llibreria Pytorch
train = datasets.FashionMNIST('../data', train=True, download=True, transform=transform)
test = datasets.FashionMNIST('../data', train=False, transform=transform)
# Transformam les dades en l'estructura necessaria per entrenar una xarxa
train_loader = torch.utils.data.DataLoader(train, train_batch_size)
test_loader = torch.utils.data.DataLoader(test, test_batch_size)

## Definició de la xarxa

### Feina a fer

1. Definir la primera xarxa convolucional. A continuació teniu una llista de les capes que podeu emprar:


- `Conv2d`: Capa convolucional en 2 dimensions. Com a paràmetres principals trobarem:

  - in_channels: canals d'entrada.
  - out_channels : canals de sortida (nombre de filtres).
  - kernel_size: mida del filtre.
  - stride: desplaçament del filtre. Típicament pren per valor 1.
  - padding: ampliació de la imatge per evitar pèrdua de dimensionalitat.

- `MaxPool2d`: Capa de max pooling. Aquesta capa no té paràmetres entrenables. Però si:

  - kernel_size: Mida del filtre del qual es seleccionarà el màxim.
  - stride: desplaçament del filtre.

- `Dropout`: Dropout és un mètode de regularització (evitar `overfitting`) que aproxima l'entrenament d'un gran nombre de xarxes neuronals amb diferents arquitectures en paral·lel. Durant l'entrenament, una part de les sortides de la capa s'ignoren aleatòriament o s'abandonen. Això té l'efecte de fer que la capa sembli i es tracti com una capa amb un nombre diferent de nodes i connectivitat a la capa anterior. En efecte, cada actualització d'una capa durant l'entrenament es realitza amb una vista diferent de la capa configurada. Hem d'especificar quines capes tenen `dropout` de manera individual. Té un únic paràmetre amb valor per defecte $p=0.5$ Els valors típics d'aquest paràmetre varien entre $0.5$ i $0.8$.


- `Linear`

- `ReLU`


2. Per posibilitar la visualització de les imatges passades per les capes convolucionals farem que funció `forward`tengui diverses sortides (diferents valors de `return`) un per cadda capa convolucional de la xarxa.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #= nn.Conv2d(in_channels= ¿?, out_channels=¿?, kernel_size=¿?, stride=¿?, padding=¿?)
        # = nn.MaxPool2d(kernel_size=¿?, stride=2)


    def forward(self, x):
        # TODO
        output = F.log_softmax(x, dim=1)
        return output

## Entrenament

Això no varia massa de la setmana anterior

### Feina a fer

1. Modificar la sortida de la xarxa, ara retorna diversos valors, encara que aquí només us interessa un.

In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target, reduction='sum') 
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Average: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), loss.item()/ len(data)))
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum') 
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max probability
            correct += pred.eq(target.view_as(pred)).sum().item()
 
  
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

A continuació definim els paràmetres d'entrenament i el bucle principal:

### Adam

Aquesta setmana introduirem un nou algorisme d'optimització anomenat `Adam`. Fins ara hem emprat el descens del gradient (`SGD`). 

`Adam()` és un algorisme d'optimització amplament emprat, tal com el descens del gradient, és iteratiu. A la literatura trobam arguments que indiquen que, tot i que Adam convergeix més ràpidament, SGD  generalitza millor que Adam i, per tant, resulta en un rendiment final millor. 

[Més info](https://medium.com/geekculture/a-2021-guide-to-improving-cnns-optimizers-adam-vs-sgd-495848ac6008)


### Feina a fer:
1. Mostrar el nombre de paràmetres de la xarxa (també a la xarxa de la setmana passada)
```
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
```
2. Dibuixar els gràfics de la funció de pèrdua amb les dues funcions d'optimització que coneixem.

In [ ]:
use_cuda = False
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

epochs = 15
lr =0.00001

model = Net().to(device)


optimizer = #TODO

# Guardam el valor de peèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_loader)


## Resultats

Aquí visualitzarem els resultats d'aprenentatge de la xarxa. 

### Feina a fer:

1. Fer una predicció del primer _batch_ del conjunt de _test_.
2. Visualitzar una imatge del _batch_ i posar la predicció i el groun truth com a títol de la imatge.
3. Visualitzar el resultat de la mateixa imatge passada per tots els filtres de cada convolució de la vostra xarxa.
4. **Extra**: Fer la matriu de confusió de les 10 classes per poder entendre el que no estau fent bé (la xarxa no està fent bé).

A tenir en compte:

#### Subplots

Per fer graelles d'imatges podeu empar la funció `subplots`. Més [informació](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html)

#### Device

Si heu emprat _GPU_ per accelerar el procés d'entrenament, els resultats que obtenim de la xarxa també seràn a la _GPU_. **Pytorch** proporciona la funció `cpu()` que retorna una còpia d'aquest objecte a la memòria de la CPU.

#### Detach
Per poder operar amb els resultats de la predicció emprarem la funció `detach` que retorna un nou Tensor "separat" del graf (xarxa) en curs.

Per tant per transformar el tensor que retorna la xarxa en un array de la lliberia _Numpy_ caldria fer el següent:

  ```
  resultat_np = resultat.detach().numpy()
  ```
Si a més hem executat l'entrenament en _GPU_:
  ```
  resultat_np = resultat.cpu().detach().numpy()
  ```


In [ ]:
def generador(loader):
  for data, target in test_loader:
    yield data, target


#TODO 